La columna `address` de la tabla principal `g_sitios` conntiene info importante sobre como tratar los jsons de estados. ya que contiene informacion de direccion y ciudades.

si se normaliza esta columna en una tabla aparte que contenga: la ciudad la direccion y codigo postal, mas la clave unica gmap_id, podemos agrupar por estados con sus ciudades.

con un eda sobre las categorias de los negocios podemos decidir que estados usar y cuales no, basandonos en datos y no en suposiciones.

informacion: 

aqui solo hare codigo, no voy a documentar, si tienen dudas se lo explico en una reunion que hagamos.

In [18]:
import pandas as pd
import os

g_sitios = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\google_sitios.parquet', engine='pyarrow')
g_sitios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2998428 entries, 0 to 3025010
Data columns (total 18 columns):
 #   Column          Dtype  
---  ------          -----  
 0   name            object 
 1   address         object 
 2   gmap_id         object 
 3   description     object 
 4   latitude        float64
 5   longitude       float64
 6   avg_rating      float64
 7   num_of_reviews  int64  
 8   price           object 
 9   state           object 
 10  url             object 
 11  monday          object 
 12  tuesday         object 
 13  wednesday       object 
 14  thursday        object 
 15  friday          object 
 16  saturday        object 
 17  sunday          object 
dtypes: float64(3), int64(1), object(14)
memory usage: 434.6+ MB


In [19]:
address = g_sitios[['gmap_id','address', 'name']]
address.head()

,gmap_id,address,name
0,0x88f16e41928ff687:0x883dad4fd048e8f8,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",Porter Pharmacy
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",City Textile
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",San Soo Dang
3,0x80c2c89923b27a41:0x32041559418d447,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",Nova Fabrics
4,0x80c2c632f933b073:0xc31785961fe826a6,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",Nobel Textile Co


In [20]:
# Separar la columna 'address' en nuevas columnas
address_split = address['address'].str.split(',', n=3, expand=True)

# Cambiar el nombre de las columnas
address_split.columns = ['nombre', 'direccion', 'ciudad', 'cod.postal']

In [21]:
# Agregar la identificacion unica a las nuevas columnas separadas.
address = address[['gmap_id']].join(address_split)
address.head()

,gmap_id,nombre,direccion,ciudad,cod.postal
0,0x88f16e41928ff687:0x883dad4fd048e8f8,Porter Pharmacy,129 N Second St,Cochran,GA 31014
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,City Textile,3001 E Pico Blvd,Los Angeles,CA 90023
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,761 S Vermont Ave,Los Angeles,CA 90005
3,0x80c2c89923b27a41:0x32041559418d447,Nova Fabrics,2200 E 11th St,Los Angeles,CA 90021
4,0x80c2c632f933b073:0xc31785961fe826a6,Nobel Textile Co,719 E 9th St,Los Angeles,CA 90021


In [22]:
# Agregar una columna nueva llamada estados.
address['estado'] = None

# Listas de los 51 códigos postales y nombres de estados.
codigos_postales = [
    'CA', 'NY', 'IA', 'GA', 'FL', 'TX', 'LA', 'OR', 'WV', 'VA', 
    'AR', 'PA', 'NM', 'NC', 'TN', 'WI', 'NJ', 'IN', 'IL', 'DC', 
    'MD', 'ME', 'NE', 'WA', 'MI', 'OH', 'OK', 'MO', 'KS', 'UT', 
    'HI', 'NV', 'AZ', 'AL', 'CO', 'MA', 'ID', 'SC', 'RI', 'KY', 
    'AK', 'MT', 'MN', 'CT', 'MS', 'SD', 'WY', 'NH', 'DE', 'VT', 
    'ND'
]

nombres_estados = [
    'California', 'New York', 'Iowa', 'Georgia', 'Florida', 'Texas', 
    'Louisiana', 'Oregon', 'West Virginia', 'Virginia', 'Arkansas', 
    'Pennsylvania', 'New Mexico', 'North Carolina', 'Tennessee', 'Wisconsin', 
    'New Jersey', 'Indiana', 'Illinois', 'District of Columbia', 'Maryland', 
    'Maine', 'Nebraska', 'Washington', 'Michigan', 'Ohio', 'Oklahoma', 
    'Missouri', 'Kansas', 'Utah', 'Hawaii', 'Nevada', 'Arizona', 
    'Alabama', 'Colorado', 'Massachusetts', 'Idaho', 'South Carolina', 
    'Rhode Island', 'Kentucky', 'Alaska', 'Montana', 'Minnesota', 
    'Connecticut', 'Mississippi', 'South Dakota', 'Wyoming', 
    'New Hampshire', 'Delaware', 'Vermont', 'North Dakota'
]

In [23]:
# cod.postal contiene las letras del estado, podemos filtrar los estados rapidamente.
for codigo, estado in zip(codigos_postales, nombres_estados):
    address.loc[address['cod.postal'].str.contains(codigo, na=False), 'estado'] = estado

In [24]:
# El cod.postal puede ser nulo por falta de informacion ya que se corre la informacion hacia la izquierda.
# Para solucionar esto filtramos los cod.postales nulos que se corrieron hacia la izquierda
for codigo, estado in zip(codigos_postales, nombres_estados):
    address.loc[(address['cod.postal'].isnull()) & (address['ciudad'].str.contains(codigo, na=False)), 'estado'] = estado

In [25]:
# Ahora que estan corregidos las mayorias de los cod.postal algunos datos se cargaron al reves, conteniendo el cod en nombre y el nombre en cod.postal.
for estado in nombres_estados:
    address.loc[(address['estado'].isnull()) & (address['nombre'].str.contains(estado, na=False)), 'estado'] = estado

In [26]:
# Eliminar filas donde al menos 5 columnas son nulas
columnas = ['gmap_id', 'nombre', 'direccion', 'ciudad', 'cod.postal', 'estado']
address = address.dropna(thresh=len(columnas) - 1)

In [27]:
# Ultimas condicionales: el cod.postal no es nulo, pero su codigo aun se encuentra en la ciudad.
for codigo, estado in zip(codigos_postales, nombres_estados):
    address.loc[(address['cod.postal'].notnull()) & (address['ciudad'].str.contains(codigo, na=False)), 'estado'] = estado

# El cod.postal no es nulo, pero esta escrito el estado en la ciudad.  
for codigo, estado in zip(codigos_postales, nombres_estados):
    address.loc[(address['cod.postal'].notnull()) & (address['ciudad'].str.contains(estado, na=False)), 'estado'] = estado
    
# El cod.postal contiene escrito el estado y no un codigo.  
for codigo, estado in zip(codigos_postales, nombres_estados):
    address.loc[(address['cod.postal'].notnull()) & (address['cod.postal'].str.contains(estado, na=False)), 'estado'] = estado

In [28]:
# Terminada la normalizacion solo quedaron muy pocas filas donde no se puede saber su estado. Se eliminaran.
address = address.dropna(subset=['estado'])
address = address.drop(columns=['nombre'])

# Agregamos un identificador unico a cada estado para que se conecte con la tabla principal sitios.
address['id_estado'] = address['estado'].factorize()[0] + 1
address['id_estado'] = address['id_estado'].astype('int32')

In [ ]:
# Mezclamos la informacion con la tabla principal.
g_sitios = g_sitios.merge(address[['gmap_id', 'id_estado']], on='gmap_id', how='left')
g_sitios = g_sitios.drop(columns=['address'])
g_sitios = g_sitios.drop(columns=['state']) # Informacion estatica. No sirve
g_sitios = g_sitios.rename(columns={'gmap_id': 'id_negocio'})

# Borrar filas con id_estado nulos, ya que no van a tener informacion para conectarse con reviews sin un estado.
g_sitios = g_sitios.dropna(subset=['id_estado'])
g_sitios['id_estado'] = g_sitios['id_estado'].astype('int32') # tipo de dato entero.

In [29]:
# Terminado la informacion. guardamos el dataframe en parquet.
os.makedirs('../../../data', exist_ok=True)
address.to_parquet('../../../data/google_address.parquet', index=False)

# Muestra parcial de la tabla:
address.head(5)

,gmap_id,direccion,ciudad,cod.postal,estado,id_estado
0,0x88f16e41928ff687:0x883dad4fd048e8f8,129 N Second St,Cochran,GA 31014,Georgia,1
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,3001 E Pico Blvd,Los Angeles,CA 90023,California,2
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,761 S Vermont Ave,Los Angeles,CA 90005,California,2
3,0x80c2c89923b27a41:0x32041559418d447,2200 E 11th St,Los Angeles,CA 90021,California,2
4,0x80c2c632f933b073:0xc31785961fe826a6,719 E 9th St,Los Angeles,CA 90021,California,2


In [ ]:
# agregar lat y lon a address
# borrar lat y lon de g_sitios.

In [31]:
# Terminado la informacion. guardamos el dataframe principal actualizado en parquet.
#os.makedirs('../../../data', exist_ok=True)
#g_sitios.to_parquet('../../../data/google_sitios_act.parquet', index=False)